In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.table import Table, join, QTable, vstack
import astropy.units as u
import sys
import pyneb as pn
from multiprocessing import Pool
import multiprocessing as mp
import math
from astropy.io import fits
from orcs.process import SpectralCube

from astropy.nddata import NDData, Cutout2D
from astropy.wcs import WCS

import astropy.units as u
from astropy.coordinates import SkyCoord

from reproject import reproject_interp
from regions import PixCoord

import pylab as pl

### Import data

In [ ]:
galaxynum = 5
galdic = {1:'NGC4254', 2:'NGC4535', 3:'NGC3351', 4:'NGC2835', 5:'NGC0628'}  #There is no SITELLE data for NGC 4254, NGC 2835 has the best data 
galaxy = galdic[galaxynum]
print(galaxy)

### Import SITELLE data cube
infile = f"/home/habjan/jupfiles/data/{galaxy}_cube.hdf5"
cube = SpectralCube(infile);

### Import MUSE image for reprojection
hdul = fits.open(f"/home/habjan/jupfiles/data/{galaxy}_IMAGE_FOV_Johnson_B_WCS_Pall_mad.fits")
muse_data = NDData(data=hdul['DATA'].data, mask=np.isnan(hdul['DATA'].data), meta=hdul['DATA'].header, wcs=WCS(hdul['DATA'].header))    
muse_data.data[muse_data.data==0] = np.nan

### Import Halpha Flux masks
hdul = fits.open(f"/home/habjan/jupfiles/data/{galaxy}_MAPS.fits")
Halpha = NDData(data=hdul['HA6562_FLUX'].data, mask=np.isnan(hdul['HA6562_FLUX'].data), meta=hdul['HA6562_FLUX'].header, wcs=WCS(hdul['HA6562_FLUX'].header))
Halpha.data[muse_data.data==0] = np.nan

### Import HII region spatial masks for locations of each HII region
hdul = fits.open(f"/home/habjan/jupfiles/data/{galaxy}_nebulae_mask_V2.fits")        #nebulae_mask_V2 , HIIreg_mask
nebulae_mask = NDData(data = hdul[0].data.astype(float), mask=Halpha.mask, meta=hdul[0].header, wcs=WCS(hdul[0].header)) 
nebulae_mask.data[nebulae_mask.data==-1] = np.nan

### Import FITS cube to use WCS information
hdul = fits.open(f"/home/habjan/jupfiles/data/{galaxy}_cube.fits")
header = hdul[0].header
wcs = WCS(header,naxis=2)

### Import SITELLE deepframe image to reprject HII region mask
hdul = fits.open(f"/home/habjan/jupfiles/data/{galaxy}_deepframe.fits")
deepframe = NDData(data=hdul[0].data, meta=hdul[0].header, wcs=wcs)

### Import PHANGS Nebular catalog
infile = open("/home/habjan/jupfiles/data/Nebulae_catalogue_v3.fits",'rb')     #Nebulae_catalogue_v3 , HIIregion_cat_DR2_native
hdul = Table.read(infile)
musedata = hdul[hdul['gal_name'] == f'{galaxy}']

### Make a list of HII region pixel locations

In [ ]:
regions = []
pixamount = []

for i in range(len(musedata)):
    testcoord = WCS(nebulae_mask.meta).pixel_to_world(np.where(nebulae_mask.data == i)[1], np.where(nebulae_mask.data == i)[0])
    if len(testcoord) == 0:
        regions.append((np.array([]), np.array([])))
        continue
    sitpix = np.transpose(cube.world2pix((testcoord.ra.degree, testcoord.dec.degree)))
    sitpix = sitpix[0].astype(np.int64), sitpix[1].astype(np.int64)
    regions.append(sitpix)
    pixamount.append(len(sitpix[0]))

batches = 5
batch = math.ceil(len(musedata)/batches)

reglist = [regions[j:j+batch] for j in range(0, len(musedata), batch)]

### Obtain wavelength array

In [ ]:
wavespec = cube.extract_integrated_spectrum(reglist[0][0])
wave = np.array(np.real(wavespec[0]), dtype=np.float64)

### Define a function to extract a spectrum from the SITELLE cube for a given galaxy

In [ ]:
def spectra(sitcube, reg, inwave):

    try:
        spec = sitcube.extract_integrated_spectrum(reg)
        flux = np.real(spec[1])/10**-20
    
    except: 

        flux = np.zeros(len(inwave))
        flux[:] = np.nan

    return flux

### Multiprocessing of function

In [ ]:
if __name__ == "__main__":

    pronum = len(reglist)

    paramlist = [[(cube, reglist[j][i], wave) for i in range(len(reglist[j]))] for j in range(len(reglist))]
    
    pool = mp.Pool(processes = pronum)          #count processes are inititiated

    list1 = [pool.apply_async(spectra, args = p) for p in paramlist[0]]
    list2 = [pool.apply_async(spectra, args = p) for p in paramlist[1]]
    list3 = [pool.apply_async(spectra, args = p) for p in paramlist[2]]
    list4 = [pool.apply_async(spectra, args = p) for p in paramlist[3]]
    list5 = [pool.apply_async(spectra, args = p) for p in paramlist[4]]

results1 = [list1[i].get() for i in range(len(list1))]
results2 = [list2[i].get() for i in range(len(list2))]
results3 = [list3[i].get() for i in range(len(list3))]
results4 = [list4[i].get() for i in range(len(list4))]
results5 = [list5[i].get() for i in range(len(list5))]

### Compile multiprocessing results into a single array

In [ ]:
spectra1 = [np.array(results1[i], dtype=np.float64) for i in range(len(results1))]
spectra2 = [np.array(results2[i], dtype=np.float64) for i in range(len(results2))]
spectra3 = [np.array(results3[i], dtype=np.float64) for i in range(len(results3))]
spectra4 = [np.array(results4[i], dtype=np.float64) for i in range(len(results4))]
spectra5 = [np.array(results5[i], dtype=np.float64) for i in range(len(results5))]

spectra = np.concatenate([spectra1, spectra2, spectra3, spectra4, spectra5], dtype=np.float64)

### Sky Subtraction function

In [ ]:
def skyback(incube, nmask, phdata, ingal):

    nebpix = (np.where(~np.isnan(nmask.data))[0], np.where(~np.isnan(nmask.data))[1])
    testcoord = WCS(nmask.meta).pixel_to_world(nebpix[1], nebpix[0])
    sitpix = np.transpose(incube.world2pix((testcoord.ra.degree, testcoord.dec.degree)))
    sitpix = sitpix[0].astype(np.int64), sitpix[1].astype(np.int64)

    r = np.max([np.max(sitpix[0]) - np.min(sitpix[0]), np.max(sitpix[1]) - np.min(sitpix[1])]) / 2 
    rmin = r * 3
    if ingal == 'NGC0628':
        rmin = r * 2.5
    rmax = np.sqrt(r**2 + rmin**2)

    cenreg = np.where(np.min(phdata['deproj_dist']) == phdata['deproj_dist'])[0][0]
    nebpix = (np.where(nmask.data == cenreg)[0], np.where(nmask.data == cenreg)[1])
    testcoord = WCS(nmask.meta).pixel_to_world(nebpix[1], nebpix[0])
    sitpix = np.transpose(incube.world2pix((testcoord.ra.degree, testcoord.dec.degree)))
    sitpix = sitpix[0].astype(np.int64), sitpix[1].astype(np.int64)
    x = np.median(sitpix[0])
    y = np.median(sitpix[1])

    imin = x - rmax
    imax = x + rmax + 1
    jmin = y - rmax
    jmax = y + rmax + 1
    xlist = []
    ylist = []
    buffval = 50

    for i in np.arange(imin, imax):
        if 0 + buffval <= i <= 2048 - buffval:
            for j in np.arange(jmin, jmax):
                if 0 + buffval <= j <= 2064 - buffval:
                    ij = np.array([i,j])
                    dist = np.linalg.norm(ij - np.array((x,y)))
                    i, j = int(i), int(j)
                    distnum = 3
                    if dist > rmin and dist <= rmax:# and np.all(np.isnan(nmask.data[i-distnum:i+distnum,j-distnum:j+distnum])):
                        xlist.append(i)
                        ylist.append(j)

    inpix = (xlist, ylist)

    batchnum = 1000 
    annuluslist = [(inpix[0][i:i+batchnum], inpix[1][i:i+batchnum]) for i in range(0, len(inpix[0]), batchnum)]
    speclist = []

    for i in range(len(annuluslist)):
        try:
            spec = incube.extract_integrated_spectrum(annuluslist[i], mean_flux=True)
            speclist.append(np.real(spec[1]))
        except:
            continue

    return np.mean(speclist, axis=0) / 10**-20

### Create a list of Sky Background spectra

In [ ]:
skyspec = skyback(cube, nebulae_mask, musedata, galaxy)
skyspecs = np.array([skyspec * pixamount[i] for i in range(len(musedata))])

### Create an HDU object with the wavelengths, spectra and sky background

In [ ]:
primary_hdu = fits.PrimaryHDU(wave)
image_hdu = fits.ImageHDU(spectra)
sky_hdu = fits.ImageHDU(skyspecs)

spectab = fits.HDUList([primary_hdu, image_hdu, sky_hdu])

### Save Spectra data

In [ ]:
spectab.writeto(f'/home/habjan/jupfiles/data/{galaxy}_SITELLE_Spectra.fits', overwrite=True)  #, overwrite=True